## Imports

Clone and install https://github.com/MTG/pycompmusic

Download data using `download-dunya.py`

In [ ]:
import json, os, sys
import pickle, csv
import time
import datetime
from collections import OrderedDict
import itertools
from copy import deepcopy

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import plotly.express as px

import scipy
import librosa
from IPython.display import Audio as ipy_audio

In [ ]:
from quicktranscribe import tonic, pitch, wave

## Parse + Plot

In [ ]:
DATA_DIR = "/Users/neerajaabhyankar/Repos/icm-shruti-analysis/data-dunya-hindustani/"

In [ ]:
def plot_annotations(track):
    ctonic = tonic.read_tonic(DATA_DIR + track + ".ctonic.txt")
    pitch_annotations, aps = pitch.read_pitch(DATA_DIR + track + ".pitch.txt")
    
    pv = pitch.PitchValidator(audio_array=np.array([]), sampling_rate=0)
    pv.set_annotation(pitch_annotations=pitch_annotations, annotation_rate=aps)
    pv.set_tonic(ctonic)
    
    pv.plot_annotations_hist()

## Compare

In [ ]:
plot_annotations("Omkar Dadarkar - Raag Todi")

In [ ]:
plot_annotations("Omkar Dadarkar - Raag Multani")

In [ ]:
plot_annotations("Kaustuv Kanti Ganguli - Raag Puriya Dhanashree")

In [ ]:
plot_annotations("Kaustuv Kanti Ganguli - Raag Shree")